In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import src.dab_util as du

# Align Original / Recored Files

Using Txgain 81

In [ ]:
#path = "../test_dat/out.iq"
path = "./recored.dat"
frame = 96*8192
offset = 5 * frame + 8192
length  = 2 * frame

In [ ]:
sig_rec = np.fromfile(path, dtype=np.complex64, count=offset + length)[offset:]

In [ ]:
#sig_rec = sig_rec[30370+0*frame:30370+1*frame]

In [ ]:
i = 0
l = length
plt.plot(np.abs(sig_rec[i:i + l]))

In [ ]:
n = sig_rec.shape[0]
max_mean = np.mean(np.sort(np.abs(sig_rec))[int(n*0.90):])

In [ ]:
sym = int(frame / 76.0)
mask = np.ones((int(4*sym)))
mask[int(2*sym) : int(3*sym)] = -1

In [ ]:
import scipy.signal as ss

In [ ]:
c = ss.correlate(np.abs(sig_rec), mask, mode='same')

In [ ]:
max1 = np.argmax(c[:frame])
max2 = frame + np.argmax(c[frame:])
max2 - max1 - frame < sym / 100

In [ ]:
plt.plot(c)
plt.plot((max1, max1), (0, 1200))
plt.plot((max2, max2), (0, 1200))

In [ ]:
i = max2
l = 2*sym
plt.plot(np.abs(sig_rec[i:i + l]))

In [ ]:
sig_rec.tofile("./orig_frame_8.iq")

In [ ]:
path = "../test_dat/out.iq"
frame = 96*8192
offset = 8 * frame + 8192 + 10500 + 72
length  = 1 * frame

In [ ]:
sig_orig = np.fromfile(path, dtype=np.complex64, count=offset + length)[offset:]

In [ ]:
plt.plot(sig_orig)

In [ ]:
sig_orig.tofile("./orig_frame_8.iq")

In [ ]:
sig_rec = sig_rec / sig_rec.std()
sig_orig = sig_orig / sig_orig.std()

In [ ]:
i = 10000
l = 400
plt.plot(sig_orig[i:i+l], label="in")
plt.plot(sig_rec[i:i+l], label="out")
plt.legend()

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

# Approximate Amplifier Function

In [ ]:
plt.plot(np.abs(sig_orig), np.abs(sig_rec), 'o')
plt.xlabel("Amplitude In")
plt.ylabel("Amplitude Out")

deg = 8
t = np.linspace(0,5)
model = make_pipeline(PolynomialFeatures(deg), Ridge())
model.fit(np.abs(sig_orig.reshape(-1,1)), np.abs(sig_rec.reshape(-1,1)))
y = model.predict(t.reshape(-1,1))
plt.plot(t, np.abs(y))

# Approximate Inverse Function

In [ ]:
plt.plot(np.abs(sig_orig), np.abs(sig_rec), 'o')
plt.xlabel("Amplitude In")
plt.ylabel("Amplitude Out")

deg = 8
t = np.linspace(0,3)
model = make_pipeline(PolynomialFeatures(deg), Ridge(alpha=100))
model.fit(np.abs(sig_rec.reshape(-1,1)),np.abs(sig_orig.reshape(-1,1)) )
y = model.predict(t.reshape(-1,1))
plt.plot(t, np.abs(y))

# Phase Error

In [ ]:
l = 100000
plt.plot(np.abs(sig_orig[i:i+l]), (np.angle(sig_rec[i:i+l], deg = 1) - np.angle(sig_orig[i:i+l], deg = 1)) % 180, 'o')
plt.xlabel("Amplitude In")
plt.ylabel("Phase difference")